In [177]:
import numpy as np
import pandas as pd
import random
import os.path
import math

from scipy.stats import multivariate_normal as binormal
from scipy.sparse.linalg import eigs

import matplotlib.pyplot as plt

In [178]:
def n_cross_val(A, n):
    for i in range(0, len(A), n):
        yield (A[:i]+A[i+n:], A[i:i + n])

In [179]:
class FLDA:
    def __init__(self,X,y):
        self.X = X
        self.y = y
        self.obs, self.n = X.shape
        assert(self.obs==y.shape[0])
        self.classes=np.unique(y)
        self.nclass=len(self.classes)
    
    def build(self):
        self.Xeach = self._split_by_class(self.X, self.y)
        self._compute_all_mean()
        Sw,Sb=self._calculate_scatter()
        # print("Sw.shape:", Sw.shape)
        # print("Sb.shape:", Sb.shape)
        evals,evecs= eigs( np.dot(np.linalg.inv(Sw), Sb), k=2, which='LM' )
        return np.dot(self.X, evecs)
            
    def _calculate_scatter(self):
        # within class scatter
        Sw = np.zeros((self.n, self.n))
        diff = self.Xeach[0] - np.array(list(self.mueach[0])*(self.Xeach[0].shape[0])).reshape(self.Xeach[0].shape)
        for i in range(1, self.nclass):
            c=self.classes[i]
            diff=np.vstack( (diff,
                        self.Xeach[c] - np.array(list(self.mueach[c])*(self.Xeach[c].shape[0])).reshape(self.Xeach[c].shape)) )
        diff=np.matrix(diff)
        
        assert(self.obs==diff.shape[0]), "Dimensions mismatch"
        for e in range(self.obs):
            Sw += np.dot(diff[e,:].T, diff[e,:])
            
        # between class scatter
        Sb = np.zeros((self.n, self.n))
        for c in self.classes:
            deltamu=self.mueach[c]-self.mu
            Sb += self.Xeach[c].shape[0] * np.dot(deltamu, deltamu.T)
        return (Sw,Sb)
        
    def _split_by_class(self, X, y):
        return {c:X[np.where(y==c)[0],:] for c in self.classes}
    
    def _compute_all_mean(self):
        self.mueach={c:np.mean(self.Xeach[c], axis=0).reshape(self.n,-1) for c in self.classes}
        self.mu=np.mean(self.X, axis=0).reshape([self.n,-1])
    

In [180]:
class MultiVariateGaussian:
    def __init__(self, X, y):
        self.X = X+np.random.normal(0,0.001, (X.shape))
        self.y = y
        self.obs, self.n = X.shape
        assert(self.obs==y.shape[0])
        self.classes=np.unique(y)
        self.nclass=len(self.classes)
        
    def _predict_score(self, Xnew):
        posterior = np.zeros((Xnew.shape[0], self.nclass))
        for i,c in enumerate(self.classes):
            m=self.mu[c].flatten()
            sigma=self.sigma[c]
            posterior[:,i]=binormal.logpdf(Xnew, m, sigma)
        logprior = [np.log(p) for p in self.prior]
        posterior += logprior
        return posterior

    def _predict_class(self, score):
        pred = np.argmax(score, axis=1)
        return np.array([self.classes[i] for i in pred])
        
    def predict(self, Xt):
        score = self._predict_score(Xt)
        return self._predict_class(score)
        
    def calc_error(self, ynew, target):
        correctness = np.array([yn == y for (yn,y) in zip(ynew, target)])
        return 1 - (np.sum(correctness) / target.size)
    
    def validate(self, Xt, yt):
        Xt += np.random.normal(0,0.001, (Xt.shape))
        mypredictions=self.predict(Xt)
        return self.calc_error(mypredictions, yt)
        
    def _train(self):
        self._compute_mean_and_sigma()
        self.prior = [np.sum(self.y == c) / self.y.size for c in self.classes]
    
    def build(self):
        self.Xeach=self._split_by_class(self.X, self.y)
        self._train()
        return self
        
    def _split_by_class(self, X, y):
        return {c:X[np.where(y==c)[0],:] for c in self.classes}
    
    def _compute_mean_and_sigma(self):
        self.mu={c:np.mean(self.Xeach[c], axis=0).reshape(self.n,-1) for c in self.classes}
        self.sigma={c:np.cov(self.Xeach[c],bias=1, rowvar=0) for c in self.classes}
    

In [195]:
def LDA2dGaussGM(filename, num_crossval):
    assert os.path.isfile(filename) and os.access(filename, os.R_OK)
    df=pd.read_csv(filename, sep=',', header = None)
    
    data = df.as_matrix()
    X=data[:, :-1]
    y=data[:, -1]
    del df, data
    
    X=X+np.random.normal(0, 0.0001, X.shape) #to prevent numerical problem
    
    if len(np.unique(y))>15:
        # if the target values are more than some reasonable no (15), we take that as binary classifier
        b = np.percentile(y, 50)
        f=np.vectorize(lambda x: 0 if x<b else 1)
        y=f(y)
        assert(X.shape[0]==y.shape[0])
        y=np.reshape(y, [X.shape[0], 1])
       
    # project into Rx2 dimensions
    W=FLDA(X,y).build()
    #print(W.shape)
    
    indices = list(range(len(y)))
    random.shuffle(indices)
    
    E=[]
    fold=1
    for (train, test) in n_cross_val(indices, len(indices)//num_crossval):
        Y_train, Y_test, X_train, X_test = y[train], y[test], X[train], X[test]
        
        # Bi-variate Gaussian classifier
        gaussian=MultiVariateGaussian(X_train,Y_train).build()
        ee=gaussian.validate(X_test, Y_test)
        print("Test error-rate for fold-%s:"%fold, ee)
        fold+=1
        E.append(ee)
    errmu = np.mean(E)*100
    errsigma = np.std(E)
    
    print()
    print("Mean test-error %0.2f percent"%errmu)
    print("Std test-error %0.2f percent"%errsigma)
        

In [196]:
LDA2dGaussGM('digits.csv', 10)

Test error-rate for fold-1: 0.0167597765363
Test error-rate for fold-2: 0.0335195530726
Test error-rate for fold-3: 0.0446927374302
Test error-rate for fold-4: 0.0614525139665
Test error-rate for fold-5: 0.0502793296089
Test error-rate for fold-6: 0.0614525139665
Test error-rate for fold-7: 0.0391061452514
Test error-rate for fold-8: 0.0335195530726
Test error-rate for fold-9: 0.0223463687151
Test error-rate for fold-10: 0.0782122905028
Test error-rate for fold-11: 0.0

Mean test-error 4.01 percent
Std test-error 0.02 percent
